In [1]:
import warnings
warnings.filterwarnings("ignore")

from scipy import constants as cst
from pint import UnitRegistry
ureg = UnitRegistry()
ureg = UnitRegistry(autoconvert_offset_to_baseunit = True)
import sys
import multiprocessing
import subprocess
import numpy as np
import shutil
import os

path_to_code = "generated-codes/chapter7/"
sys.path.append(path_to_code)

from MinimizeEnergy import MinimizeEnergy
from MonteCarlo import MonteCarlo
from WriteLAMMPSfiles import write_topology_file, write_lammps_parameters, write_lammps_variables

"The values of the potential constants for argon have been taken throughout as  
E*/k= 119.76°K, r*=3.822 A, v*=23.79 cm3/mole, as determined by Michels6 from  
second virial coefficient data." [1]  

[1] Wood and Parker. The Journal of Chemical Physics, 27(3):720–733, 1957.  

In [2]:
kB = cst.Boltzmann*ureg.J/ureg.kelvin # boltzman constant
Na = cst.Avogadro/ureg.mole # avogadro
R = kB*Na # gas constant

In [3]:
def launch_MC_code(tau):

    epsilon = (119.76*ureg.kelvin*kB*Na).to(ureg.kcal/ureg.mol) # kcal/mol
    r_star = 3.822*ureg.angstrom # angstrom
    sigma = r_star / 2**(1/6) # angstrom
    N_atom = 200 # no units
    m_argon = 39.948*ureg.gram/ureg.mol
    T = (55 * ureg.degC).to(ureg.degK) # 55°C
    # volume_star = (23.79 * ureg.centimeter**3/ureg.mole).to(ureg.angstrom**3/ureg.mole)
    volume_star = r_star**3 * Na * 2**(-0.5)
    cut_off = sigma*2.5
    displace_mc = sigma/5 # angstrom
    volume = N_atom*volume_star*tau/Na
    box_size = volume**(1/3)
    folder = "outputs_tau"+str(tau)+"/"

    em = MinimizeEnergy(maximum_steps=100,
                        thermo_period=10,
                        dumping_period=10,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    em.run()

    mc = MonteCarlo(maximum_steps=20000,
        dumping_period=1000,
        thermo_period=1000,
        neighbor=50,
        displace_mc = displace_mc.magnitude,
        desired_temperature = T.magnitude,
        number_atoms=[N_atom],
        epsilon=[epsilon.magnitude], 
        sigma=[sigma.magnitude],
        atom_mass=[m_argon.magnitude],
        box_dimensions=[box_size.magnitude,
                        box_size.magnitude,
                        box_size.magnitude],
        initial_positions = em.atoms_positions*em.reference_distance,
        cut_off=cut_off.magnitude,
        data_folder=folder,
    )
    mc.run()

    folder = "lammps_tau"+str(tau)+"/"
    if os.path.exists(folder) is False:
        os.mkdir(folder)

    write_topology_file(mc, filename=folder+"initial.data")
    write_lammps_parameters(mc, filename=folder+"PARM.lammps")
    write_lammps_variables(mc, filename=folder+"variable.lammps")

    mycwd = os.getcwd() # initial path
    os.chdir(folder)
    shutil.copyfile("../lammps/input.lmp", "input.lmp")
    subprocess.call(["/home/simon/Softwares/lammps-2Aug2023/src/lmp_serial", "-in", "input.lmp"])
    os.chdir(mycwd)

In [4]:
if __name__ == "__main__":
    tau_values = np.round(np.logspace(-0.126, 0.882, 10),2)
    pool = multiprocessing.Pool()
    squared_numbers = pool.map(launch_MC_code, tau_values)
    pool.close()
    pool.join()

step epot maxF
step epot maxFstep epot maxF
0 5643296580373.397 576800541039197.625step epot maxF
step epot maxF0 634893986983097.000 154986401065073440.000step epot maxF
step epot maxF
0 97359009544394.109 16408166885839386.000


0 14706148090892.330 1765262278576218.000
0 2158039270016.191 188967458318514.781
step epot maxF0 249214717919256.719 50339439601637440.000
0 37882987690666.391 5368345169376455.000

step epot maxF
0 1602768034558115.000 476703787033699392.000


0 3931027684326804.000 1430587670129549568.000step epot maxF

0 9675772265341772.000 4361071283023053312.000
10 5407032.178 319368727.827
10 9174076.962 471949118.952
10 57530532.445 1342150842.63610 84587513.757 1829477184.029

10 104857998.329 2580766936.817
10 16874647.872 683168608.163
10 134145248.832 3661487543.431
10 32165151.472 966528846.47110 208002708.580 4656725584.836

10 415909967.035 7464076572.955
20 -32.084 15.211
20 -35.456 19.860
20 169.088 120.27420 386.167 139.602

20 -18.380 38.656
20 1459.835 38